In [12]:
%pip install anthropic
%pip install rich
%pip install python-dotenv

ERROR: Invalid requirement: 'anthropic,': Expected end or semicolon (after name and no valid version specifier)
    anthropic,
             ^
Note: you may need to restart the kernel to use updated packages.


In [19]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from typing import Dict, List

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_completion(messages: List[Dict[str, str]]) -> str:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_completion_tokens=4092, 
    )
    return response.choices[0].message.content

In [8]:
import re
from typing import Dict, List
from pathlib import Path

class MarkdownSectionParser:
    def __init__(self):
        self.section_pattern = r'^#+ .*$'  # '#'으로 시작하는 헤더 패턴
        self.end_sections = {'ACKNOWLEDGEMENTS', 'REFERENCES', 'CONCLUSION', 'CONCLUSIONS'}
        
    def parse_sections(self, markdown_path: str) -> Dict[str, str]:
        sections = {}
        current_section = None
        current_content = []
        
        with open(markdown_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            
        for line in lines:
            if re.match(self.section_pattern, line):
                section_title = line.strip('# \n')
                
                if current_section and current_content:
                    sections[current_section] = ''.join(current_content).strip()
                
                # 대문자로 변환하여 비교
                if any(end_sec in section_title.upper() for end_sec in self.end_sections):
                    break
                    
                current_section = section_title
                current_content = []
            else:
                if current_section is None and 'ABSTRACT' not in line.upper():
                    continue
                current_content.append(line)
        
        # 마지막 섹션 저장
        if current_section and current_content:
            sections[current_section] = ''.join(current_content).strip()
        
        # Abstract 필터링
        filtered_sections = {}
        include_section = False
        
        for section, content in sections.items():
            if 'ABSTRACT' in section.upper():
                include_section = True
            
            if include_section:
                filtered_sections[section] = content
                
        return filtered_sections

In [3]:
from anthropic import Anthropic
from typing import Dict, List
import time
from tqdm import tqdm

class PaperExplainer:
    def __init__(self):
        self.conversation_history = []
        self.delay = 1  # API 호출 간 딜레이 (초)
        
    def _create_section_prompt(self, section_title: str, section_content: str, is_first: bool = False) -> str:
        if is_first:
            return f"""You are an expert academic paper explainer. Please explain the following section '{section_title}' 
            from an academic paper in a clear and concise manner. Please explain in Korean.

            Section content:
            {section_content}"""
        else:
            return f"""Based on our previous discussion of the paper, please explain the following section '{section_title}'.
            
            Section content:
            {section_content}"""
    
    def explain_section(self, section_title: str, section_content: str) -> str:
        try:
            # Create prompt based on whether this is the first section
            is_first = len(self.conversation_history) == 0
            prompt = self._create_section_prompt(section_title, section_content, is_first)
            
            # Add previous conversation for context
            self.conversation_history.append({"role": "user", "content": prompt})
            
            response = get_completion(self.conversation_history)
            
            # Update conversation history
            self.conversation_history.append({"role": "assistant", "content": response})            
            # API 호출 간 딜레이
            time.sleep(self.delay)
            
            return response
            
        except Exception as e:
            print(f"Error explaining section {section_title}: {str(e)}")
            return f"Error: Failed to explain section {section_title}"

    def explain_paper(self, sections: Dict[str, str]) -> Dict[str, str]:
        """
        논문의 각 섹션을 순차적으로 설명
        
        Args:
            sections: 섹션 제목과 내용을 매핑한 딕셔너리
            
        Returns:
            섹션 제목과 설명을 매핑한 딕셔너리
        """
        explanations = {}
        
        print("\nProcessing sections:")
        for title, content in tqdm(sections.items(), desc="Explaining sections"):
            print(f"\nProcessing: {title}")
            explanation = self.explain_section(title, content)
            explanations[title] = explanation
            
        return explanations

In [42]:

from typing import Optional, List
class PaperQA:
    def __init__(self, context: Optional[List[Dict[str, str]]] = None):
        self.conversation_history = context or []
        self.delay = 1
        
    def load_paper_context(self, explanations: Dict[str, str]):
        """논문 설명을 대화 기록에 로드"""
        context = "Here's the paper summary:\n\n"
        for section, explanation in explanations.items():
            context += f"## {section}\n{explanation}\n\n"
            
        # 논문 컨텍스트를 대화 기록에 추가
        self.conversation_history.append({"role": "user", "content": context})

    def ask_question(self, question: str) -> str:
        """논문에 대한 질문에 답변"""
        try:
            # 질문 프롬프트 생성
            prompt = f"""Based on the paper we discussed, please answer the following question in Korean. 
            Be specific and cite relevant sections when possible.

            Question: {question}"""
            
            # 이전 대화 기록과 함께 질문 전송
            self.conversation_history.append({"role": "user", "content": prompt})
            
            # Claude에 질문
            response = get_completion(self.conversation_history)
            
            # 대화 기록 업데이트
            self.conversation_history.append({"role": "assistant", "content": response})
            
            time.sleep(self.delay)
            return response
            
        except Exception as e:
            print(f"Error processing question: {str(e)}")
            return f"Error: Failed to process question"
    
    def view_conversation_history(self, start_idx: int = 0, end_idx: Optional[int] = None) -> None:
        """대화 내역을 출력하는 함수
        
        Args:
            start_idx: 시작 인덱스 (기본값: 0)
            end_idx: 종료 인덱스 (기본값: None, None일 경우 끝까지 출력)
        """
        # 논문 컨텍스트는 제외하고 실제 대화만 출력
        conversations = [
            msg for msg in self.conversation_history 
            if not msg["content"].startswith("Here's the paper summary:")
        ]
        
        # end_idx가 None이면 리스트 끝까지
        end_idx = end_idx if end_idx is not None else len(conversations)
        
        print("\n=== 대화 내역 ===\n")
        for i, msg in enumerate(conversations[start_idx:end_idx], start=start_idx):
            role = msg["role"].upper()
            if role == "ASSISTANT":
                print(f"\n🤖 Assistant ({i}):\n{msg['content']}\n")
                print("-" * 80)
            elif role == "USER":
                print(f"\n👤 User ({i}):\n{msg['content']}\n")
                print("-" * 80)
    
    
    def get_last_n_conversations(self, n: int = 1) -> None:
        """최근 n개의 대화 내역을 출력
        
        Args:
            n: 출력할 최근 대화 개수 (기본값: 1)
        """
        conversations = [
            msg for msg in self.conversation_history 
            if not msg["content"].startswith("Here's the paper summary:")
        ]
        start_idx = max(0, len(conversations) - n)
        self.view_conversation_history(start_idx)

In [5]:
from typing import Tuple

def process_paper(markdown_path: str) -> Tuple[Dict[str, str], PaperQA]:
    """
    전체 논문 처리 프로세스
    """
    # 1. Markdown 파싱
    parser = MarkdownSectionParser()
    sections = parser.parse_sections(markdown_path)
    
    print(sections)
    # 2. 섹션별 설명 생성
    explainer = PaperExplainer()
    explanations = explainer.explain_paper(sections)

    # 3. 질문 답변 준비 
    qa = PaperQA()
    qa.load_paper_context(explanations)

    return explanations, qa

In [47]:
from dotenv import load_dotenv
import os

load_dotenv()

markdown_path = "input_file/INSTRUCTION TUNING WITH GPT-4.md"
output_dir = "output_file"

explanations, qa = process_paper(markdown_path)

{'ABSTRACT': 'Prior work has shown that finetuning large language models (LLMs) using machinegenerated instruction-following data enables such models to achieve remarkable zero-shot capabilities on new tasks, and no human-written instructions are needed. In this paper, we present the first attempt to use GPT-4 to generate instructionfollowing data for LLM finetuning. Our early experiments on instruction-tuned LLaMA models show that the 52K English and Chinese instruction-following data generated by GPT-4 leads to superior zero-shot performance on new tasks to the instruction-following data generated by previous state-of-the-art models. We also collect feedback and comparison data from GPT-4 to enable a comprehensive evaluation and reward model training. We make our data generated using GPT-4 as well as our codebase publicly available. 1', 'INTRODUCTION': 'Large Language Models (LLMs) have shown impressive generalization capabilities such as incontext-learning (Brown et al., 2020) and c

Explaining sections:   0%|          | 0/10 [00:00<?, ?it/s]


Processing: ABSTRACT


Explaining sections:  10%|█         | 1/10 [00:04<00:41,  4.59s/it]


Processing: INTRODUCTION


...

Explaining sections:  80%|████████  | 8/10 [01:13<00:19,  9.76s/it]


Processing: 4.3 COMPARISONS WITH SOTA USING AUTOMATIC EVALUATION


Explaining sections:  90%|█████████ | 9/10 [01:26<00:10, 10.52s/it]


Processing: 5 RELATED WORK


Explaining sections: 100%|██████████| 10/10 [01:44<00:00, 10.47s/it]


In [48]:
output_dir = "output_file"

input_filename = Path(markdown_path).stem  # 파일 이름만 추출 (확장자 제외)
output_path = os.path.join(output_dir, f"{input_filename}_explained.md")

with open(output_path, 'w', encoding='utf-8') as f:
    for section, explanation in explanations.items():
        f.write(f"\n## {section}\n\n")
        f.write(explanation)
        f.write("\n\n---\n")

print(f"\nExplanations saved to: {output_path}")


Explanations saved to: output_file/INSTRUCTION TUNING WITH GPT-4_explained.md


In [49]:
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel
from rich.syntax import Syntax
from rich.table import Table
from typing import Dict
import os

class MarkdownPrinter:
    def __init__(self):
        self.console = Console()
        
    def print_markdown_file(self, file_path: str):
        """마크다운 파일을 이쁘게 출력"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                markdown_content = f.read()
            
            # 마크다운 렌더링
            md = Markdown(markdown_content)
            
            # 마크다운 내용 출력
            self.console.print(md)
            
        except Exception as e:
            self.console.print(f"[bold red]Error reading file: {str(e)}[/]")
            
    def print_sections(self, sections: Dict[str, str]):
        """섹션별로 구분하여 출력"""
        for section, content in sections.items():
            # 섹션 제목
            self.console.print("\n")
            self.console.print(Panel(
                f"[bold cyan]{section}[/]",
                border_style="cyan"
            ))
            
            # 섹션 내용
            md = Markdown(content)
            self.console.print(md)
            
            # 구분선
            self.console.print("[dim]" + "="*80 + "[/]")

printer = MarkdownPrinter()

# 마크다운 파일 출력
printer.print_markdown_file(output_path)

ABSTRACT                                                      

이 연구에서는 대형 언어 모델(LLM)을 기계가 생성한 지시문 데이터를 사용하여 미세 조정(finetuning)하면 새로운        
작업에서도 뛰어난 제로샷(Zero-shot) 성능을 발휘할 수 있다는 것을 이전 연구가 보여주었다고 설명합니다. 즉, 인간이   
작성한 지시문이 필요 없게 됩니다. 본 논문은 GPT-4를 활용하여 LLM 미세 조정을 위한 지시문 데이터를 생성한 첫 시도를 
소개합니다. 연구 결과, GPT-4가 생성한 5만 2천 개의 영어 및 중국어 지시문 데이터가 기존 최첨단 모델들이 생성한      
지시문 데이터보다 새로운 작업에서 뛰어난 제로샷 성능을 보여주었습니다. 또한, 포괄적인 평가 및 보상 모델 훈련을 위해
GPT-4로부터 피드백 및 비교 데이터를 수집하였습니다. 우리는 GPT-4를 사용하여 생성한 데이터와 코드베이스를 공개하고  
있습니다.                                                                                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                   INTRODUCTION                                                    

이 논문은 대형 언어 모델(LLM)의 뛰어난 일반화 능력을 설명하며, 이러한 모델들이 자연어 지시를 따르고 실제 세상에서의
작업을 완료할 수 있도록 하는 방법을 탐구하고 있습니다. 그 방법 중 하나로 지시문 튜닝(instruction-tuning)이 있으며, 
이는 인간이 주석을 단 프롬프트와 피드백을 사용하여 모델을 다양한 작업에 맞게 미세 조정하거나, 수동 또는 자동으로   
생성된 지시문을 추가한 공개 벤치마크와 데이터세트를 사용하는 방식으로 구현됩니다.                                  

특히 Self-Instruct 튜닝은 최신의 지시문 튜닝된 교사 LLM이 생성한 데이터를 학습하여, LLM을 인간 의도에 맞추는       
간단하고 효과적인 방법입니다. 이 연구 방향은 LLM의 제로샷 및 소수샷 일반화 능력을 향상시키는 효과적인 수단을 제공해
왔습니다. 최근 ChatGPT와 GPT-4의 성공은 이러한 지시문 튜닝을 통해 오픈소스 LLM을 개선할 큰 기회를 제공합니다.      
LLaMA는 이러한 오픈소스 LLM들 중 하나로, 상용 LLM인 GPT-3와 성능을 견줄 수 있습니다. LLaMA가 지시를 따를 수 있도록 
하려면, 뛰어난 성능과 저비용의 Self-Instruct 튜닝이 빠르게 채택되고 있습니다.                                      

본 논문에서 우리는 처음으로 GPT-4를 Self-Instruct 튜닝의 교사로 사용하는 방법을 제안합니다. 우리의 주요 기여는     
다음과 같습니다:                                                                                                   

 • GPT-4 데이터: 우리는 영어와 중국어로 된 52K 개의 지시문 데이터셋과 세 개의 지시문 튜닝된 모델의 출력을 평가한   
   피드백 데이터를 포함하여 GPT-4가 생성한 데이터를 공개합니다.                                                    
 • 모델 및 평가: GPT-4가 생성한 데이터를 기반으로 지시문 튜닝된 LLaMA 모델과 보상 모델을 개발했습니다. 우리는 세   
   가지 기준에 대한 인간 평가, GPT-4 피드백을 사용한 자동 평가, 그리고 비정형 지시문에 대한 ROUGE-L 점수를 사용하여
   지시문 튜닝된 LLM의 품질을 평가합니다.                                                                          

이 연구는 GPT-4로 생성된 데이터를 사용한 LLM 지시문 튜닝의 효과성을 확인하고, 일반 목적의 지시문을 따르는          
에이전트를 구축하는 데 있어 실용적인 팁을 제안합니다.                                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                     2 DATASET                                                     

이 논문의 '2 데이터셋' 섹션에서는 데이터 수집 과정과 GPT-4를 사용한 데이터 생성 방법에 대해 설명합니다.            

데이터 수집: Alpaca 데이터셋(Taori et al., 2023)에서 수집된 52,000개의 고유한 지시문을 재사용합니다. 각 지시문은   
모델이 수행해야 할 작업을 설명합니다. 지시문에는 작업에 대한 선택적 맥락이나 입력이 있을 수도 있고 없을 수도       
있습니다. Alpaca 데이터셋에서는 GPT-3.5(text-davinci-003)를 사용해 출력을 생성했지만, 우리는 GPT-4(gpt-4)를        
활용하여 출력을 생성합니다. GPT-4를 사용해 다음 네 가지 데이터셋을 생성합니다.                                     

 1 영어 지시문-답변 데이터: Alpaca에서 수집한 52,000개의 지시문 각각에 대해 GPT-4가 영어로 답변을 제공합니다.      
   자세한 내용은 알고리즘 1에 나와 있습니다. GPT-4와 Self-Instruct를 사용하여 자체 지시 세트를 구성하는 반복적     
   과정을 향후 작업으로 남겨둡니다.                                                                                
 2 중국어 지시문-답변 데이터: ChatGPT를 사용하여 52,000개의 지시문을 중국어로 번역하고, GPT-4가 중국어로 답변하게  
   합니다. 이를 통해 LLaMA에 기반한 중국어 지시문-따르기 모델을 구축하고, 지시문 튜닝의 언어 간 일반화 능력을      
   연구할 수 있습니다.                                                                                             
 3 비교 데이터: GPT-4에게 자신의 응답을 1부터 10까지 평가하도록 요청합니다. 또한, GPT-4에게 GPT-4, GPT-3.5, 그리고 
   OPT-IML을 포함한 세 모델의 응답을 비교하여 평가하도록 요청합니다. 이는 보상 모델을 훈련하는 데 사용됩니다.      
 4 비정상 지시문에 대한 답변: 68,000개의 지시문-입력-출력 삼중 코어 데이터셋에 대해 GPT-4의 답변을 디코딩합니다. 이
   하위 세트는 GPT-4와 우리 지시문 튜닝된 모델들 간의 차이를 계량화하는 데 사용됩니다.                             

데이터 통계: 그림 1에서는 GPT-4와 GPT-3.5의 영어 출력 응답 세트를 비교합니다. 각 출력에 대해 동사 뿌리와 직접      
목적어 명사를 추출하고, 각 출력 세트에서 고유한 동사-명사 쌍의 빈도를 계산합니다. 빈도가 10 이상인 동사-명사 쌍은  
그림 1(a)와 (b)에 표시되며, 두 세트의 가장 빈번한 25개의 쌍을 그림 1(c)에 비교합니다. 출력 시퀀스 길이의 빈도     

In [43]:
question = """ 
원본 데이터 셋을 샘플링해서 입력 시퀀스에 조건화를 한 후 새로운 데이터를 생성하는 것 같은데 이게 맞아? 그리고 조건화가 뭐임?
"""

qa = PaperQA(qa.conversation_history)
response = qa.ask_question(question)
print(response)

네, 맞습니다. ReST에서 새로운 데이터를 생성하는 과정은 원본 데이터셋을 샘플링하여 입력 시퀀스를 선택하고, 이를 기반으로 조건화된 새로운 출력 시퀀스를 생성하는 방식으로 이루어집니다. 여기서 "조건화"라는 것은 주어진 입력 시퀀스 \(\pmb{x}\)에 대해 그 조건 하에서 가능한 출력 시퀀스 \(\pmb{y}\)를 생성한다는 의미입니다.

조건화는 확률적 모델링에서 매우 중요한 개념인데, 이는 주어진 입력이 어떤 특정한 상태나 값일 때의 출력 확률을 뜻합니다. 언어 모델링에서는 입력 시퀀스(또는 문맥)가 주어졌을 때 다음 가능한 출력 시퀀스를 생성하는 과정을 말합니다. 이 과정은 자동 회귀 모델을 사용하여 구현되며, 모델이 이미 학습한 확률 분포에 기반해 \(\pi_{\theta}(\pmb{y}|\pmb{x})=\prod_{t=1}^{T}\pi_{\theta}(y_{t}|\pmb{y}_{1:t-1}, \pmb{x})\) 형태로 다음 토큰 \(y_t\)를 계산합니다.

ReST에서는 이를 통해 현재 정책 \(\pi_{\theta}\)로 입력 시퀀스에 맞는 새로운 출력 시퀀스를 생성하고, 그 결과를 데이터셋에 추가하여 데이터를 확장시킵니다. 이러한 방식은 모델이 이미 알고 있는 문맥에 기반해 추가적인 학습 데이터를 생성할 수 있도록 하며, 이는 Grow 단계에서 원본 데이터셋의 샘플을 이용해 새로운 시퀀스를 생성하는 과정에서 핵심적인 역할을 합니다. 이는 Section 3의 내용을 바탕으로 설명한 것입니다.
